# Train and deploy on Kubeflow from Notebooks

This notebook introduces you to using Kubeflow Fairing to train and deploy a model to Kubeflow on AWS backend. This notebook demonstrate how to:
 
* Train an XGBoost model in a local notebook,
* Use Kubeflow Fairing to train an XGBoost model remotely on Kubeflow using some of the supported backends,
* Use Kubeflow Fairing to deploy a trained model to Kubeflow, and
* Call the deployed endpoint for predictions.

To learn more about how to run this notebook locally, see the guide to [training and deploying on GCP from a local notebook][gcp-local-notebook].

[gcp-local-notebook]: https://kubeflow.org/docs/fairing/gcp-local-notebook/

## Set up your notebook for training an XGBoost model

Import the libraries required to train this model.

In [1]:
!pip install --user -r requirements.txt

     |████████████████████████████████| 6.7 MB 61.7 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 4.8 MB/s  eta 0:00:01
     |████████████████████████████████| 71 kB 335 kB/s  eta 0:00:01
     |████████████████████████████████| 80 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 59 kB 12.0 MB/s eta 0:00:01
  Created wheel for Flask-OpenTracing: filename=Flask_OpenTracing-1.1.0-py3-none-any.whl size=11453 sha256=2d774f440725223d001a040cb94b6d85d5c92b3e01bf52c630ae1410785b1050
  Stored in directory: /home/jovyan/.cache/pip/wheels/ad/4b/2d/24ff0da0a0b53c7c77ce59b843bcceaf644c88703241e59615
  Created wheel for jaeger-client: filename=jaeger_client-4.1.0-py3-none-any.whl size=65467 sha256=7c0397640cc578c997ed65f835c3997b86f393ce6664fb034f59cc2dfa457b2c
  Stored in directory: /home/jovyan/.cache/pip/wheels/e9/9b/8c/503d0cc13b39a551c054515683ba1d15b40324c863dc442e66
  Created wheel for opentra

In [2]:
import argparse
import logging
import joblib
import sys
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor

In [3]:
logging.basicConfig(format='%(message)s')
logging.getLogger().setLevel(logging.INFO)

Define a function to split the input file into training and testing datasets.

In [4]:
def read_input(file_name, test_size=0.25):
    """Read input data and split it into train and test."""
    data = pd.read_csv(file_name)
    data.dropna(axis=0, subset=['SalePrice'], inplace=True)

    y = data.SalePrice
    X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])

    train_X, test_X, train_y, test_y = train_test_split(X.values,
                                                      y.values,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)

Define functions to train, evaluate, and save the trained model.

In [5]:
def train_model(train_X,
                train_y,
                test_X,
                test_y,
                n_estimators,
                learning_rate):
    """Train the model using XGBRegressor."""
    model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate)

    model.fit(train_X,
            train_y,
            early_stopping_rounds=40,
            eval_set=[(test_X, test_y)])

    print("Best RMSE on eval: %.2f with %d rounds" %
               (model.best_score,
                model.best_iteration+1))
    return model

def eval_model(model, test_X, test_y):
    """Evaluate the model performance."""
    predictions = model.predict(test_X)
    logging.info("mean_absolute_error=%.2f", mean_absolute_error(predictions, test_y))

def save_model(model, model_file):
    """Save XGBoost model for serving."""
    joblib.dump(model, model_file)
    logging.info("Model export success: %s", model_file)

Define a class for your model, with methods for training and prediction.

In [6]:
class HousingServe(object):
    
    def __init__(self):
        self.train_input = "ames_dataset/train.csv"
        self.n_estimators = 50
        self.learning_rate = 0.1
        self.model_file = "trained_ames_model.dat"
        self.model = None

    def train(self):
        (train_X, train_y), (test_X, test_y) = read_input(self.train_input)
        model = train_model(train_X,
                          train_y,
                          test_X,
                          test_y,
                          self.n_estimators,
                          self.learning_rate)

        eval_model(model, test_X, test_y)
        save_model(model, self.model_file)

    def predict(self, X, feature_names=None):
        """Predict using the model for given ndarray."""
        if not self.model:
            self.model = joblib.load(self.model_file)
        # Do any preprocessing
        prediction = self.model.predict(data=X)
        # Do any postprocessing
        return prediction

## Train an XGBoost model in a notebook

Call `HousingServe().train()` to train your model, and then evaluate and save your trained model.

In [8]:
HousingServe().train()

[18:27:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:177514
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161858
[2]	validation_0-rmse:147237
[3]	validation_0-rmse:134132
[4]	validation_0-rmse:122224
[5]	validation_0-rmse:111538
[6]	validation_0-rmse:102142
[7]	validation_0-rmse:93392.3
[8]	validation_0-rmse:85824.6
[9]	validation_0-rmse:79667.6
[10]	validation_0-rmse:73463.4
[11]	validation_0-rmse:68059.4
[12]	validation_0-rmse:63350.5
[13]	validation_0-rmse:59732.1
[14]	validation_0-rmse:56260.7
[15]	validation_0-rmse:53392.6
[16]	validation_0-rmse:50770.8
[17]	validation_0-rmse:48107.8
[18]	validation_0-rmse:45923.9
[19]	validation_0-rmse:44154.2
[20]	validation_0-rmse:42488.1
[21]	validation_0-rmse:41263.3
[22]	validation_0-rmse:40212.8
[23]	validation_0-rmse:39089.1
[24]	validation_0-rmse:37691.1
[25]	validation_0-rmse:36875.2
[26]	validation_0-r

mean_absolute_error=18173.15
Model export success: trained_ames_model.dat


## Set up Kubeflow Fairing for training and predictions

Import the `fairing` library and configure the backend environment that your training or prediction job will run in.

In [9]:
from kubeflow import fairing

# Default to using the GKE backend.
# Change to `KubeflowAzureBackend` or `KubeflowAWSBackend`,etc if you are on other cloud providers.
# Appropriately modify settings below for the corresponding backend.
FAIRING_BACKEND = 'KubeflowAWSBackend'
DOCKER_REGISTRY="958854133286.dkr.ecr.us-east-1.amazonaws.com"

## Train an XGBoost model remotely on Kubeflow

Import the `TrainJob` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the training data, and the training job's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the training job on Kubeflow.

In [13]:
from kubeflow.fairing.backends import KubeflowAWSBackend
from kubeflow.fairing import TrainJob
train_job = TrainJob(HousingServe, input_files=['ames_dataset/train.csv', "requirements.txt"],
                     docker_registry=DOCKER_REGISTRY,
                     backend=KubeflowAWSBackend())
train_job.submit()

Using default base docker image: registry.hub.docker.com/library/python:3.6.8
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_z1j1pz9u
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Not able to find aws credentials secret: aws-secret
Waiting for fairing-builder-m4slw-sg9fm to start...
Waiting for fairing-builder-m4slw-sg9fm to start...
Waiting for fairing-builder-m4slw-sg9fm to start...
Pod started running True


INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0000] Built cross stage deps: map[]
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0001] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0001] Unpacking rootfs as cm

Not able to find aws credentials secret: aws-secret
The job fairing-job-sptpx launched.
Waiting for fairing-job-sptpx-hsjvm to start...
Waiting for fairing-job-sptpx-hsjvm to start...
Waiting for fairing-job-sptpx-hsjvm to start...
Pod started running True


[0]	validation_0-rmse:177565.32812
Will train until validation_0-rmse hasn't improved in 40 rounds.
[1]	validation_0-rmse:161967.20312
[2]	validation_0-rmse:148001.90625
[3]	validation_0-rmse:135010.15625
[4]	validation_0-rmse:123514.68750
[5]	validation_0-rmse:113210.38281
[6]	validation_0-rmse:103914.61719
[7]	validation_0-rmse:95352.98438
[8]	validation_0-rmse:87878.78125
[9]	validation_0-rmse:81683.14062
[10]	validation_0-rmse:75828.78906
[11]	validation_0-rmse:70085.50781
[12]	validation_0-rmse:65076.06641
[13]	validation_0-rmse:60899.81641
[14]	validation_0-rmse:57354.23047
[15]	validation_0-rmse:54106.52734
[16]	validation_0-rmse:51402.43359
[17]	validation_0-rmse:48774.05078
[18]	validation_0-rmse:46360.19141
[19]	validation_0-rmse:44304.82812
[20]	validation_0-rmse:42618.65234
[21]	validation_0-rmse:41219.89062
[22]	validation_0-rmse:39885.14453
[23]	validation_0-rmse:38977.96484
[24]	validation_0-rmse:37856.48047
[25]	validation_0-rmse:36739.77734
[26]	validation_0-rmse:35847

Cleaning up job fairing-job-sptpx...


'fairing-job-sptpx'

## Deploy the trained model to Kubeflow for predictions

Import the `PredictionEndpoint` and use the configured backend class. Kubeflow Fairing packages the `HousingServe` class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [15]:
from kubeflow.fairing import PredictionEndpoint
endpoint = PredictionEndpoint(HousingServe, input_files=['trained_ames_model.dat', "requirements.txt"],
                              service_type='LoadBalancer',
                              docker_registry=DOCKER_REGISTRY,
                              backend=KubeflowAWSBackend())
endpoint.create()

Using default base docker image: registry.hub.docker.com/library/python:3.6.8
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_kjl1x90e
/home/jovyan/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Not able to find aws credentials secret: aws-secret
Waiting for fairing-builder-ks79f-vbwfk to start...
Waiting for fairing-builder-ks79f-vbwfk to start...
Waiting for fairing-builder-ks79f-vbwfk to start...
Pod started running True


INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Resolved base name registry.hub.docker.com/library/python:3.6.8 to registry.hub.docker.com/library/python:3.6.8
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0000] Built cross stage deps: map[]
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0000] Error while retrieving image from cache: getting file info: stat /cache/sha256:e2b625c433e2e3c9a72eb92483c7e6ebe32163e320258f6a60badc44d9eb2806: no such file or directory
INFO[0000] Downloading base image registry.hub.docker.com/library/python:3.6.8
INFO[0001] Unpacking rootfs as cm

Deploying the endpoint.


Waiting for prediction endpoint to come up...


Cluster endpoint: http://None:5000/predict
Prediction endpoint: http://None:5000/predict


## Call the prediction endpoint

Create a test dataset, then call the endpoint on Kubeflow for predictions.

In [ ]:
(train_X, train_y), (test_X, test_y) = read_input("ames_dataset/train.csv")
endpoint.predict_nparray(test_X)

## Clean up the prediction endpoint

Delete the prediction endpoint created by this notebook.

In [ ]:
endpoint.delete()